# Model 1
## Dataset:

## Programmer: Giovanni Vecchione
## Date: 4/4/24
## Subject: Machine Learning 2 - Project 4
Use Recurrent Neural Networks(RNN)/LSTM/GRU or sentiment analysis to build the project. 
Submit your project as Jupyter notebook